In [128]:
import yfinance as yf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from tqdm import tqdm

### Question 1: Withdrawn IPOs by Company Type
What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list [stockanalysis.com/ipos/withdrawn](stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Definition of terms
* **IPO withdrawal:** In the world of stock analytics, this refers to a situation where a company that has filed to go public (i.e., launched an Initial Public Offering) decides to cancel or postpone the offering before its shares are officially listed and traded on a stock exchange.

In [46]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url, headers=headers)

ipo_withdraw = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\3236582231.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_withdraw = pd.read_html(response.text)


In [47]:
ipo_withdraw_dfs = ipo_withdraw[0]
ipo_withdraw_dfs.head(2)

,Symbol,Company Name,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",-,-


In [48]:
ipo_withdraw_dfs.loc[0]

Symbol                              UNFL
Company Name      Unifoil Holdings, Inc.
Price Range                $3.00 - $4.00
Shares Offered                   2000000
Name: 0, dtype: object

In [49]:
ipo_withdraw_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          99 non-null     object
 1   Company Name    99 non-null     object
 2   Price Range     99 non-null     object
 3   Shares Offered  99 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [58]:
# IPO value will the thw average of price range * shared offer
def split_price_range(x):
    if x == '-' or pd.isna(x):
        return 0.0
    
    x = x.replace('$', '').replace(' ', '')
    if '-' in x:
        low, high = map(float, x_lst)
        return (low + high) / 2
    
    return float(x)

In [55]:
ipo_withdraw_dfs['Price Per Share'] = ipo_withdraw_dfs['Price Range'].apply(split_price_range)
ipo_withdraw_dfs['Shares Offered'] = ipo_withdraw_dfs['Shares Offered'].apply(split_price_range)
ipo_withdraw_dfs['IPO Price'] = ipo_withdraw_dfs['Price Per Share'] * ipo_withdraw_dfs['Shares Offered']

In [79]:
ipo_price_order = ipo_withdraw_dfs.sort_values('IPO Price', ascending=False).reset_index(drop=True)
ipo_price_order['IPO Price'] = round(ipo_price_order['IPO Price'] / 1000000, 2).astype(str) + 'M'

max_ipo_price = ipo_price_order.loc[0]
max_ipo_price

Symbol                         NVL
Company Name          Novelis Inc.
Price Range        $18.00 - $21.00
Shares Offered          45000000.0
Price Per Share               19.5
IPO Price                   877.5M
Name: 0, dtype: object

### Question 2: Median Sharpe Ratio for 2024 IPOs (First 5 Months)
What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

Data Source: Using the same approach as in Question 1, download the IPOs in 2024 from:
https://stockanalysis.com/ipos/2024/

#### Definition of terms
* **Volatility:** this is a measure of how much a stock’s price moves up and down over time.
* **Sharpe Ratio:** this is a financial metric used to evaluate the risk-adjusted return of an investment. It tells you how much excess return you’re getting for the extra volatility you endure by holding a riskier asset. Example: if rist rate is 4.5%, positive Sharpe means growth exceeding the risk-free rate of 4.5%.

In [109]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_stock = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\2197623396.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_stock = pd.read_html(response.text)


In [110]:
ipo_stock_df = ipo_stock[0]

ipo_stock_df.head(2)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$3.55,-12.25%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$18.32,346.00%


In [111]:
ipo_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [119]:
# change data type
# remove dollar sign
def clean_special_chars(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].apply(lambda x: 0 if x == '-' else x)
        if c == 'Return':
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('%', '')
        else:
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('$', '')
        

def change_type_float(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].astype(float)

def change_type_date(col):
    ipo_stock_df[col] = pd.to_datetime(ipo_stock_df[col])


In [120]:
# clean cols
cols = ['IPO Price', 'Current', 'Return']
clean_special_chars(cols)
change_type_float(cols)
change_type_date('IPO Date')

In [124]:
trim_ipo_stock_df = ipo_stock_df[ipo_stock_df['IPO Date'] < '2024-06-01'].reset_index(drop=True)
trim_ipo_stock_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",NaN,14.18,NaN
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,35.09,114.06
2,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,20.41,4.50
3,2024-05-17,RFAI,RF Acquisition Corp II,10.00,10.50,5.10
4,2024-05-15,JDZG,JIADE Limited,4.00,0.27,-92.63


In [163]:
trim_ipo_stock_df.shape

(77, 6)

**Daily Stock Data**

In [166]:
risk_free_rate = 0.045

first_ticker = trim_ipo_stock_df.Symbol[0]

stock_df = yf.download(tickers = first_ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_df['Ticker'] = first_ticker
stock_df['Growth 252'] = stock_df['Close'] / stock_df['Close'].shift(252)

# how much stock prices move up and down
stock_df['Returns'] = stock_df['Close'].pct_change() # daily returns
# I could have calculated volatility on daily returns if I want
stock_df['Volatility (Yearly)']  = stock_df['Close'].rolling(30).std() * 252**0.5  


[*********************100%***********************]  1 of 1 completed


In [168]:
stock_df = stock_df.reset_index()
stock_df.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly)
251,2025-06-03,18.360001,18.360001,22.003000,16.700001,21.500000,1336500,NAKA,NaN,-0.137623,143.730280
252,2025-06-04,17.129999,17.129999,18.860001,16.510000,18.129999,770800,NAKA,5.672185,-0.066994,142.127837
253,2025-06-05,16.020000,16.020000,17.874001,15.430000,17.150000,672700,NAKA,6.022556,-0.064799,139.767994
254,2025-06-06,15.880000,15.880000,17.200001,15.550000,16.260000,431200,NAKA,5.438356,-0.008739,136.860031
255,2025-06-09,13.350000,13.350000,15.920000,12.785000,15.900000,972800,NAKA,4.890110,-0.159320,133.213858


In [170]:
for ticker in tqdm(trim_ipo_stock_df.Symbol[1:]):
    next_stock_df = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
    next_stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    next_stock_df['Ticker'] = ticker
    next_stock_df['Growth 252'] = next_stock_df['Close'] / next_stock_df['Close'].shift(252)
    # how much stock prices move up and down
    next_stock_df['Returns'] = next_stock_df['Close'].pct_change() # daily returns
    # I could have calculated volatility on daily returns if I want
    next_stock_df['Volatility (Yearly)']  = next_stock_df['Close'].rolling(30).std() * 252**0.5  

    next_stock_df = next_stock_df.reset_index()
    stock_df = pd.concat([stock_df, next_stock_df], ignore_index=True)

[*********************100%***********************]  1 of 1 completed                                                       | 0/76 [00:00<?, ?it/s]
[*********************100%***********************]  1 of 1 completed                                               | 1/76 [00:00<00:55,  1.35it/s]
[*********************100%***********************]  1 of 1 completed                                               | 2/76 [00:01<00:53,  1.40it/s]
[*********************100%***********************]  1 of 1 completed                                               | 3/76 [00:02<00:52,  1.38it/s]
[*********************100%***********************]  1 of 1 completed                                               | 4/76 [00:02<00:54,  1.31it/s]
[*********************100%***********************]  1 of 1 completed                                               | 5/76 [00:03<00:56,  1.27it/s]
[*********************100%***********************]  1 of 1 completed                                               | 6

In [171]:
stock_df.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly)
23259,2025-06-03,3.41,3.41,3.530,3.160,3.36,67200,ROMA,5.262346,0.014881,10.986046
23260,2025-06-04,3.76,3.76,3.850,3.457,3.50,271100,ROMA,5.829458,0.102639,11.789832
23261,2025-06-05,3.57,3.57,4.135,3.360,3.74,264600,ROMA,6.144579,-0.050532,12.173744
23262,2025-06-06,3.70,3.70,3.950,3.630,3.65,84400,ROMA,6.156406,0.036415,12.553202
23263,2025-06-09,2.86,2.86,3.955,2.600,3.75,381700,ROMA,4.627832,-0.227027,12.238182


In [172]:
stock_df['Sharpe'] = (stock_df['Growth 252'] - risk_free_rate) / stock_df['Volatility (Yearly)']

**Explore trading on `2025-06-06`**

In [173]:
stocke_df_20250606 = stock_df[stock_df['Date'] == '2025-06-06']
stocke_df_20250606[['Growth 252', 'Sharpe']].describe()

,Growth 252,Sharpe
count,73.000000,73.000000
mean,1.223993,0.297260
std,1.478959,0.523296
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.082241
75%,1.446667,0.331967
max,8.097413,2.835668


* `73` out of `77` companies traded on `2025-06-06`.
* Median of `Growth 252` is `0.76` (indicating a `24%` decline) while mean is `1.22` showing bias towards high growth companies (outliers), pushing the average upward.
* The dedian of Sharp Ratio is `0.08`. A positive Sharpe Ratio median for these `73` stocks growth exceeding the risk-free rate of `4.5%`.

Now lets observer the top 10 companies by growth_252d and by Sharpe

In [179]:
stocke_df_20250606.sort_values(by='Growth 252', ascending=False).reset_index(drop=True).loc[:10]

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly),Sharpe
0,2025-06-06,5.320000,5.320000,5.355000,5.289000,5.310000,7700,JL,8.097413,0.009488,14.221305,0.566222
1,2025-06-06,3.700000,3.700000,3.950000,3.630000,3.650000,84400,ROMA,6.156406,0.036415,12.553202,0.486840
2,2025-06-06,15.880000,15.880000,17.200001,15.550000,16.260000,431200,NAKA,5.438356,-0.008739,136.860031,0.039408
3,2025-06-06,7.420000,7.420000,8.650000,6.610000,6.810000,10186200,UMAC,4.966533,0.138037,11.681615,0.421306
4,2025-06-06,31.190001,31.190001,31.559000,28.350000,29.150000,3244100,NNE,4.655224,0.097081,57.122852,0.080707
5,2025-06-06,97.910004,97.910004,103.000000,95.000000,102.800003,6569700,RBRK,3.184065,-0.006696,164.768721,0.019051
6,2025-06-06,37.520000,37.520000,37.799999,37.009998,37.540001,4036300,AS,2.478203,0.003745,86.819697,0.028026
7,2025-06-06,35.480000,35.480000,35.580002,34.755001,35.070000,1715900,AHR,2.376423,0.014584,21.657313,0.107651
8,2025-06-06,10.135000,10.135000,10.470000,10.120000,10.470000,51200,SUPX,2.346065,-0.004420,8.637976,0.266389
9,2025-06-06,42.270000,42.270000,43.020000,42.126999,43.009998,460000,MRX,2.259220,-0.004006,30.146280,0.073449


In [180]:
stocke_df_20250606.sort_values(by='Sharpe', ascending=False).reset_index(drop=True).loc[:10]

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly),Sharpe
0,2025-06-06,10.600,10.600,10.600,10.600,10.600,0,BKHA,1.045881,0.000000,0.352961,2.835668
1,2025-06-06,10.850,10.850,10.850,10.850,10.850,228100,JVSA,1.071076,0.000000,0.502601,2.041531
2,2025-06-06,10.620,10.620,10.620,10.620,10.620,0,LEGT,1.049407,0.000000,0.517665,1.940267
3,2025-06-06,10.420,10.420,10.420,10.420,10.420,100,IBAC,1.044611,-0.001916,0.610592,1.637119
4,2025-06-06,0.390,0.390,0.417,0.376,0.390,109900,NCI,0.572687,0.000000,0.446672,1.181375
5,2025-06-06,10.890,10.890,10.890,10.750,10.800,383400,HLXB,1.061404,0.000000,0.904682,1.123493
6,2025-06-06,1.340,1.340,1.350,1.300,1.310,51800,MNDR,1.030769,0.007519,1.011840,0.974234
7,2025-06-06,10.800,10.800,10.820,10.780,10.780,2500,DYCQ,1.059863,0.007463,1.046983,0.969321
8,2025-06-06,0.668,0.668,0.670,0.605,0.633,31400,INTJ,0.661386,0.079160,0.827906,0.744512
9,2025-06-06,5.320,5.320,5.355,5.289,5.310,7700,JL,8.097413,0.009488,14.221305,0.566222


### Question 3: Fixed Months Holding Strategy
What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

**Goal:**
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.